<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ALPHAFOLD3_GEMINI3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. SETUP & IMPORTS (Includes all fixes) ---
print("--- 1. Setting up Environment (Installing compatible Kaleido 0.2.1) ---")
# This installation line ensures all dependencies (Biopython, Plotly, Kaleido fix)
# and the Gemini SDK are available.
!pip install biopython plotly kaleido==0.2.1 google-genai pillow -q

--- 1. Setting up Environment (Installing compatible Kaleido 0.2.1) ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.8 MB/s eta 0:00:00


In [4]:

# Necessary imports
import os
import warnings
from google.colab import userdata
from google import genai
from google.genai import types # REQUIRED FOR CONFIGURATION
from PIL import Image
import plotly.graph_objects as go
from Bio.PDB import PDBList, PDBParser
from Bio.PDB.PDBExceptions import PDBConstructionWarning

# Suppress harmless Biopython warnings
warnings.filterwarnings('ignore', category=PDBConstructionWarning)

# --- 2. API KEY RETRIEVAL AND CONFIGURATION ---
GEMINI_API_KEY = None
try:
    # Set the API key using the user's preferred multi-source retrieval logic
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Define ThinkingLevel as requested
class ThinkingLevel:
    LOW = "LOW"
    HIGH = "HIGH"


THINKING_LEVEL_SETTING=ThinkingLevel.HIGH

# Placeholder function for the 'tools' parameter (required for execution)
# Since we cannot define external functions in a simple cell,
# this will likely be the source of a NameError or an API rejection.
def run_simulation(protein_id: str):
    """Placeholder for a function to run a protein folding simulation."""
    return f"Simulation results for {protein_id} are complex."
# Model Configuration
MODEL_ID = 'gemini-3-pro-preview'

# 3. PDB DATA DEFINITIONS
PDB_ID = "6M0J"  # SARS-CoV-2 Spike RBD / ACE2 complex
PDB_FILE_NAME = f"pdb{PDB_ID.lower()}.ent"
IMAGE_FILE_NAME = f"{PDB_ID}_structure.png"

# 4. DOWNLOAD AND PROCESS PDB FILE
print(f"\n--- 2. Downloading Complex PDB ID: {PDB_ID} ---")
pdbl = PDBList()
pdbl.retrieve_pdb_file(PDB_ID, pdir='.', file_format='pdb', overwrite=True)

print("--- 3. Extracting Coordinates ---")
parser = PDBParser()
structure = parser.get_structure(PDB_ID, PDB_FILE_NAME)

xs, ys, zs, names, colors = [], [], [], [], []
color_map = {'A': 'red', 'E': 'blue'} # A: Human ACE2 (Red), E: Viral Spike RBD (Blue)

for model in structure:
    for chain in model:
        for residue in chain:
            if 'CA' in residue:
                atom = residue['CA']
                coord = atom.get_coord()
                xs.append(coord[0])
                ys.append(coord[1])
                zs.append(coord[2])

                chain_id = chain.id
                colors.append(color_map.get(chain_id, 'gray'))
                names.append(f"{chain_id}: {residue.get_resname()} {residue.get_id()[1]}")

# 5. VISUALIZATION AND STATIC IMAGE EXPORT
print("--- 4. Launching Interactive Plotly Viewer and Saving Static Image ---")
trace = go.Scatter3d(x=xs, y=ys, z=zs, mode='markers+lines',
                     marker=dict(size=4, color=colors, opacity=0.8),
                     line=dict(color='gray', width=1), hoverinfo='text', text=names)

fig = go.Figure(data=[trace])
fig.update_layout(title=f'3D Structure of COVID Spike RBD / ACE2 Complex ({PDB_ID})', scene=dict(aspectmode='data'))

fig.show(renderer="colab")
fig.write_image(IMAGE_FILE_NAME)
print(f"Structure saved as {IMAGE_FILE_NAME} for multimodal analysis.")

# --- 6. GEMINI MULTIMODAL API CALL (With Deep Think) ---
print(f"\n--- 5. Executing Gemini Multimodal Call using Model: {MODEL_ID} (Thinking Level: {THINKING_LEVEL_SETTING}) ---")

# Data Variables (Assumed to be created by preceding visualization steps)
IMAGE_FILE_NAME = "6M0J_structure.png"
PDB_ID = "6M0J"

# --- API EXECUTION BLOCK (EXACTLY AS REQUESTED) ---
print(f"\n--- Executing Final Request for Model: {MODEL_ID} (With Config Parameters) ---")

if GEMINI_API_KEY:
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)

        # NOTE: prompt_history is not defined in a single cell,
        # so we will use the current prompt, 'prompt', as it contains the multimodal data.
        prompt_text = (
            "Analyze this 3D protein complex structure. The red cluster is the Human ACE2 receptor, and the blue cluster is the Viral Spike RBD. "
            "The overall PDB ID is 6M0J. Based on this visual and the color-coding, generate a professional summary (two paragraphs) that describes:"
            "1) The crucial biological conflict shown (Red vs. Blue) and the significance of the visible binding interface."
            "2) The relevance of analyzing this structure to AI-driven drug discovery (AlphaFold 3/Isomorphic Labs)."
        )
        image_file = Image.open(IMAGE_FILE_NAME)
        prompt = [prompt_text, image_file] # Using prompt instead of prompt_history

        # --- THE EXACT REQUESTED API CALL STRUCTURE ---
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt, # Using 'prompt' as prompt_history is undefined
            config=types.GenerateContentConfig(
                # FIX 4 APPLIED: The tools argument is now inside the config object
                tools=[run_simulation],
                # FIX 3 APPLIED: Nesting thinking_level inside types.ThinkingConfig
                thinking_config=types.ThinkingConfig(
                    thinking_level=ThinkingLevel.HIGH
                )
            )
        )
        # --- END REQUESTED API CALL STRUCTURE ---

        print("\n--- GEMINI GENERATED REPORT ---")
        print(response.text)

    except Exception as e:
        print(f"\n[API EXECUTION ERROR]: The requested parameters failed the API validation or caused a NameError (due to the required function definition). Error: {e}")

else:
    print("\n[ERROR]: GEMINI_API_KEY not found. Please set your key.")

print("\n--- Integration Script Complete ---")


--- 2. Downloading Complex PDB ID: 6M0J ---
--- 3. Extracting Coordinates ---
--- 4. Launching Interactive Plotly Viewer and Saving Static Image ---


Structure saved as 6M0J_structure.png for multimodal analysis.

--- 5. Executing Gemini Multimodal Call using Model: gemini-3-pro-preview (Thinking Level: HIGH) ---

--- Executing Final Request for Model: gemini-3-pro-preview (With Config Parameters) ---

--- GEMINI GENERATED REPORT ---
Based on the visualization of the PDB structure 6M0J, here is an analysis of the protein complex:

The visual representation highlights the critical pathogenic mechanism of viral infection, depicted by the interplay between the human host and the viral invader. The red cluster (Human ACE2 receptor) functions as the cellular gateway, while the blue cluster (Viral Spike RBD) acts as the molecular "key" attempting to unlock entry into the cell. The most significant feature of this structure is the binding interface where the blue and red clusters merge. This contact zone represents a high-affinity interaction where the viral domain exploits the surface geometry of the human enzyme, creating a stable comple